In [11]:
import os
import time
import openpyxl
import pandas as pd
from typing import List
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc



load_dotenv()

True

In [12]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            logger.info('Conexão estabelecida!')
            pass
        logger.info('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        logger.info(f"Error: {e}")
        return None

In [ ]:
def generate_sql_query() -> List[str]:
    current_month = f"{datetime.now().month:02}"
    current_year = datetime.now().year % 100  # Get the last two digits of the year

    last_month = f"{(datetime.now().month - 1) % 12:02}"
    last_year = current_year if datetime.now().month > 1 else current_year - 1

    tables = [f"movprodd{last_month}{last_year}", f"movprodd{current_month}{current_year}"]

    return tables

print(generate_sql_query())

In [ ]:
conn = get_db_engine() 

def vendas_query(conn):
    query = (f"""
        SELECT 
                mprd.mprd_transacao AS transacao,
                clie.clie_cnpjcpf AS cnpj_cpf,
                clie.clie_nome AS nome_clie,
                mprd.mprd_datamvto AS data_mvto,
                mprd.mprd_numerodcto AS nfe,
                prod.prod_codbarras AS cod_barras,
                prod.prod_pesoliq AS peso,
                mprd_prod_codigo::TEXT AS cod_prod,
                TO_CHAR(mprd.mprd_qtde, '00000000000999D9999') AS qtde,
                TO_CHAR((mprd.mprd_valor / mprd.mprd_qtde), '00999D99') AS valor_unitario_2,
                TO_CHAR(mprd.mprd_vlrunitario, '00999D99') AS valor_unitario,
                mprc.mprc_vend_codigo AS cod_vend,
                mprc.mprc_codentidade AS cod_clie,
                mprd.mprd_dcto_codigo AS dcto_cod,
                prun.prun_emb AS embalagem_vend,
                SUBSTRING(clie.clie_cepres, 1,5) ||'-'|| SUBSTRING(clie.clie_cepres, 6,3) AS cep,
                mprd.mprd_unid_codigo AS unidade,
                mprd.mprd_valor AS vendas
            FROM movprodd0722 AS mprd
            LEFT JOIN movprodc AS mprc ON mprd.mprd_transacao = mprc.mprc_transacao
            LEFT JOIN produtos AS prod ON mprd.mprd_prod_codigo = prod.prod_codigo
            LEFT JOIN clientes AS clie ON mprc.mprc_codentidade = clie.clie_codigo
            WHERE mprd_status = 'N'
            AND prod.prod_marca IN ('BRF', 'BRF IN NATURA')
            AND mprd.mprd_dcto_codigo IN ('6666','6668','7339','7335','7338','7337','7260','7263','7262','7268','7264','7269','7267','7319','7318', '6680','6890')
            AND mprc.mprc_vend_codigo NOT IN ('0','00','000','0000','')
            AND mprd.mprd_vlrunitario NOT IN ('0.0100000000')
            AND clie.clie_cepres NOT IN ('00000-000','','0','00000','00000000')
            AND clie.clie_cepres > '0'
            AND clie.clie_cepres NOT IN ('')
    """)
        # AND mprd.mprd_datamvto > '2023-01-09'
    
    df = pd.read_sql_query(query, conn)
    return df

display(vendas_query(conn))

In [33]:
conn = get_db_engine() 

def vendas_query(conn):
    all_tables = ['movprodd0121', 'movprodd0221', 'movprodd0321', 'movprodd0421', 'movprodd0521', 'movprodd0621', 'movprodd0721', 'movprodd0821', 'movprodd0921', 'movprodd1021', 'movprodd1121', 'movprodd1221','movprodd0122', 'movprodd0222', 'movprodd0322', 'movprodd0422', 'movprodd0522', 'movprodd0622', 'movprodd0722', 'movprodd0822', 'movprodd0922', 'movprodd1022', 'movprodd1122', 'movprodd1222', 'movprodd0123', 'movprodd0223', 'movprodd0323', 'movprodd0423', 'movprodd0523', 'movprodd0623', 'movprodd0723', 'movprodd0823', 'movprodd0923', 'movprodd1023',  'movprodd1123', 'movprodd1223']

    dfs = []
   
    for tables in all_tables:
        query = f"""
            SELECT
                mprd.mprd_unid_codigo AS unidade,
                mprd.mprd_valor AS vendas,
                mprd.mprd_datamvto AS data_mvto
            FROM
                {tables} AS mprd
            LEFT JOIN movprodc AS mprc ON mprd.mprd_transacao = mprc.mprc_transacao
            LEFT JOIN produtos AS prod ON mprd.mprd_prod_codigo = prod.prod_codigo
            LEFT JOIN clientes AS clie ON mprc.mprc_codentidade = clie.clie_codigo
            WHERE mprd_status = 'N'
            AND prod.prod_marca IN ('BRF', 'BRF IN NATURA')
            AND mprd.mprd_dcto_codigo IN ('6666','6668','7339','7335','7338','7337','7260','7263','7262','7268','7264','7269','7267','7319','7318', '6680','6890')
            AND mprc.mprc_vend_codigo NOT IN ('0','00','000','0000','')
            AND mprd.mprd_vlrunitario NOT IN ('0.0100000000')
            AND clie.clie_cepres NOT IN ('00000-000','','0','00000','00000000')
            AND clie.clie_cepres > '0'
            AND clie.clie_cepres NOT IN ('')
            AND mprd.mprd_datamvto <= '2023-10-11'
        """
    
        df = pd.read_sql_query(query, conn)
        dfs.append(df)
    
    df_concatenado = pd.concat(dfs, ignore_index=True)
    
    df_final = df_concatenado.groupby('unidade').agg(
        vendas=('vendas', 'sum'),
        primeira_data=('data_mvto', 'min'),
        ultima_data=('data_mvto', 'max')
    ).reset_index()
    
    def format_currency(value):
        return f"R$ {value:,.2f}".replace(",", "x").replace(".", ",").replace("x", ".")
    
    df_final['vendas_formatadas'] = df_final['vendas'].apply(format_currency)
    
    return df_final

display(vendas_query(conn))

2023-10-30 15:03:09.284 | INFO     | __main__:get_db_engine:7 - Conexão estabelecida!
2023-10-30 15:03:09.285 | INFO     | __main__:get_db_engine:9 - Banco de dados conectado!
C:\Users\Marcelo Dev\AppData\Local\Temp\ipykernel_18796\2074581004.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concatenado = pd.concat(dfs, ignore_index=True)


,unidade,vendas,primeira_data,ultima_data,vendas_formatadas
0,001,30982510.70,2021-01-05,2023-10-11,"R$ 30.982.510,70"
1,002,5856958.52,2021-01-05,2023-10-11,"R$ 5.856.958,52"
2,003,4807841.13,2023-01-02,2023-10-11,"R$ 4.807.841,13"
3,010,4869505.21,2021-01-04,2022-12-30,"R$ 4.869.505,21"
